In [1]:
print("h")

h


In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import Subset
import numpy as np
from collections import defaultdict

# Define dataset path
dataset_path = r"C:\Users\Keerthana\Downloads\Leaves\Leaves"

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load full dataset
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Organize samples by class
class_indices = defaultdict(list)
for idx, (_, label) in enumerate(dataset.samples):
    class_indices[label].append(idx)

# Select exactly 500 random samples per class
selected_indices = []
for label, indices in class_indices.items():
    selected_indices.extend(np.random.choice(indices, 100, replace=False))

# Create subset dataset (only 500 images per class)
subset_dataset = Subset(dataset, selected_indices)

# Split into train (80%) and validation (20%)
train_size = int(0.8 * len(subset_dataset))
val_size = len(subset_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(subset_dataset, [train_size, val_size])

# Create DataLoaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

# Print dataset sizes
print(f"Total images per class used: 500")
print(f"Training samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")


Total images per class used: 500
Training samples: 320, Validation samples: 80


In [2]:
import timm
import torch.nn as nn
import torch

# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load MobileNetV3-Large
model = timm.create_model('mobilenetv3_large_100', pretrained=True)

# Modify the classifier for 4 classes
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 4)  # 4 output classes

# Move model to GPU
model.to(device)

print("✅ MobileNetV3 Model Loaded!")


✅ MobileNetV3 Model Loaded!


In [3]:
# Define loss function & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)


In [4]:
def train_model(model, train_loader, val_loader, epochs=3):
    model.train()
    
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()  # Reset gradients
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels)
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss:.4f}, Accuracy: {accuracy:.2f}%")

    print("✅ Training Complete!")

# Start training
train_model(model, train_loader, val_loader, epochs=3)


Epoch 1/3, Loss: 5.3208, Accuracy: 80.31%
Epoch 2/3, Loss: 0.7854, Accuracy: 97.50%
Epoch 3/3, Loss: 0.6516, Accuracy: 97.50%
✅ Training Complete!


In [5]:
def evaluate_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    print(f"Validation Accuracy: {accuracy:.2f}%")
    return accuracy

# Run evaluation
evaluate_model(model, val_loader)


Validation Accuracy: 97.50%


97.5

In [6]:
torch.save(model.state_dict(), "mobilenetv3_plant_disease.pth")
print("✅ Model saved successfully!")


✅ Model saved successfully!
